# Write predicted language into data

In [125]:
import os
os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

import torch
import pprint
import json
import data.preprocess_language_predictions as preprocess_language_predictions
from importlib import import_module
from collections import Counter

import importlib

In [119]:
class args:
    pass

# settings
args.seed = 123
args.data = '/root/data_alfred/json_feat_2.1.0'
args.split_dir = '/root/data_alfred/splits/'
args.splits = os.path.join(args.split_dir, 'apr13.json')
args.pp_folder = 'pp'
args.pframe = 300
args.fast_epoch = False

## Existing Data

In [84]:
with open(args.splits, 'r') as f:
    splits = json.load(f)

In [85]:
splits.keys()

dict_keys(['tests_seen', 'tests_unseen', 'train', 'valid_seen', 'valid_unseen', 'train_sanity'])

In [86]:
splits['train'][0]

{'repeat_idx': 0,
 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}

In [87]:
splits['valid_seen'][0]

{'repeat_idx': 0,
 'task': 'pick_heat_then_place_in_recep-PotatoSliced-None-SinkBasin-13/trial_T20190909_115736_122556'}

In [88]:
splits['tests_seen'][0]

{'repeat_idx': 0, 'task': 'trial_T20190909_042500_949430'}

## Predicted Data

In [143]:
def write_new_raw_data(splits, split_name, pred_version, high_descs_model_directory, high_descs_epoch, task_desc_model_directory, task_desc_epoch):
    
    # load predicted lang instructions
    with open('/root/data/home/hoyeung/blob_alfred_data/exp_all/{}/{}_argmax.debug_epoch_{}.preds.json'.format(high_descs_model_directory, split_name, high_descs_epoch), 'r') as f:
        pred_data_high_descs = json.load(f)
    with open('/root/data/home/hoyeung/blob_alfred_data/exp_all/{}/{}_argmax.debug_epoch_{}.preds.json'.format(task_desc_model_directory, split_name, task_desc_epoch), 'r') as f:
        pred_data_task_desc = json.load(f)
    assert set(pred_data_high_descs.keys()) - set(pred_data_task_desc.keys()) == set()
    assert set(pred_data_task_desc.keys()) - set(pred_data_high_descs.keys()) == set()

    pred_task_ids = list(pred_data_high_descs.keys())
    new_split = []
    out_paths = []
    
    if split_name == 'train_sanity':
        split_name = 'train'
    
    for task_id in pred_task_ids:
        # loop through task ids here
        root = pred_data_high_descs[task_id]['root'].replace('data/json_feat_2.1.0/', '{}/'.format(split_name))
         
        # get raw data from path
        with open(os.path.join(args.data, root, 'traj_data.json'), 'r') as f:
            raw_traj_data = json.load(f)
        
        # DANGER!
#         del(raw_traj_data['turk_annotations']['anns'][-1])
        
        next_r_idx = len(raw_traj_data['turk_annotations']['anns'])
    
        # write to raw data
        append_ann = {}
        append_ann['pred_version'] = pred_version  # e.g. v1
        high_descs = pred_data_high_descs[task_id]['p_lang_instr']
        append_ann['high_descs'] = [word.strip()+'.' for word in high_descs.split('.') if word != '']
        append_ann['task_desc'] = pred_data_task_desc[task_id]['p_lang_instr']
        raw_traj_data['turk_annotations']['anns'].append(append_ann) 

        # write out to raw data path
        out_path = os.path.join(args.data, root, 'traj_data.json')
        out_paths.append(out_path)
        
        with open(out_path, 'w') as f:
            json.dump(raw_traj_data, f)

        with open('dummy.json', 'w') as f:
            json.dump(raw_traj_data, f)
            
        # add to split
        append_split_entry = {
            'repeat_idx': next_r_idx,
            'task': root.replace(split_name+'/', '')
        }
        new_split.append(append_split_entry)
        
    with open(os.path.join(args.split_dir, '{}_{}_split.json'.format(split_name, pred_version)), 'w') as f:
        json.dump(new_split, f)
        
    with open(os.path.join(args.split_dir, '{}_{}_out_paths.json'.format(split_name, pred_version)), 'w') as f:
        json.dump(new_split, f)    
    
    return new_split, out_paths

In [144]:
train_sanity_new_split, train_sanity_raw_out_paths = write_new_raw_data(    
    splits=splits,                               
    split_name='train_sanity', 
    pred_version='v1', 
    high_descs_model_directory='model:seq2seq_nl_with_frames,name:v1_epoch_50_low_level_instrs',
    high_descs_epoch=49, 
    task_desc_model_directory='model:seq2seq_nl_with_frames,name:v1.5_epoch_50_high_level_instrs', 
    task_desc_epoch=10)

In [146]:
valid_seen_new_split, valid_seen_raw_out_paths = write_new_raw_data(    
    splits=splits,                               
    split_name='valid_seen', 
    pred_version='v1', 
    high_descs_model_directory='model:seq2seq_nl_with_frames,name:v1_epoch_50_low_level_instrs',
    high_descs_epoch=49, 
    task_desc_model_directory='model:seq2seq_nl_with_frames,name:v1.5_epoch_50_high_level_instrs', 
    task_desc_epoch=10)

In [147]:
valid_unseen_new_split, valid_unseen_raw_out_paths = write_new_raw_data(    
    splits=splits,                               
    split_name='valid_unseen', 
    pred_version='v1', 
    high_descs_model_directory='model:seq2seq_nl_with_frames,name:v1_epoch_50_low_level_instrs',
    high_descs_epoch=49, 
    task_desc_model_directory='model:seq2seq_nl_with_frames,name:v1.5_epoch_50_high_level_instrs', 
    task_desc_epoch=10)

## Save to new split file

In [149]:
def write_new_splits(path, splits):
    with open(path, 'w') as f:
        json.dump(splits, f)

In [150]:
pred_version='v1'

splits['train_sanity_{}'.format(pred_version)] = train_sanity_new_split
splits['valid_seen_{}'.format(pred_version)] = valid_seen_new_split
splits['valid_unseen_{}'.format(pred_version)] = valid_unseen_new_split

In [151]:
args.split_dir

'/root/data_alfred/splits/'

In [152]:
date = 'apr25'
write_new_splits(os.path.join(args.split_dir, '{}.json'.format(date)), splits)

In [153]:
valid_unseen_new_split

[{'repeat_idx': 3,
  'task': 'look_at_obj_in_light-CellPhone-None-FloorLamp-219/trial_T20190908_044113_026049'},
 {'repeat_idx': 3,
  'task': 'pick_heat_then_place_in_recep-Apple-None-SinkBasin-10/trial_T20190907_020258_749030'},
 {'repeat_idx': 3,
  'task': 'pick_clean_then_place_in_recep-Bowl-None-Cabinet-10/trial_T20190909_061130_844814'},
 {'repeat_idx': 3,
  'task': 'pick_and_place_with_movable_recep-Spoon-Mug-CounterTop-10/trial_T20190907_155006_533262'},
 {'repeat_idx': 3,
  'task': 'pick_and_place_simple-SaltShaker-None-Cabinet-10/trial_T20190906_191501_563086'},
 {'repeat_idx': 3,
  'task': 'pick_two_obj_and_place-PepperShaker-None-Drawer-10/trial_T20190912_221141_608117'},
 {'repeat_idx': 3,
  'task': 'pick_cool_then_place_in_recep-TomatoSliced-None-CounterTop-10/trial_T20190909_061838_159982'},
 {'repeat_idx': 3,
  'task': 'pick_heat_then_place_in_recep-Apple-None-GarbageCan-10/trial_T20190908_145356_918528'},
 {'repeat_idx': 3,
  'task': 'look_at_obj_in_light-Newspaper-None

## Purge predicted from raw data

In [ ]:
def purge_predictions(out_paths):

    for out_path in out_paths:

        with open(out_path, 'r') as f:
            traj_data = json.load(f)

        with open(out_path.replace('.json', '_backup.json'), 'w') as f:
            json.dump(traj_data, f)

        annotations = []

        for ann in raw_traj_data['turk_annotations']['anns']:
            if 'pred_version' not in ann:
                annotations.append(ann)

        raw_traj_data['turk_annotations']['anns'] = annotations

        with open(out_path, 'w') as f:
            json.dump(traj_data, f)

## Preprocess splits with existing vocab file

In [188]:
importlib.reload(preprocess_language_predictions)

<module 'data.preprocess_language_predictions' from '/root/data/home/hoyeung/alfred/data/preprocess_language_predictions.py'>

In [189]:
pred_version='v1'
splits_to_preprocess = {}
splits_to_preprocess['train_sanity_{}'.format(pred_version)] = train_sanity_new_split
splits_to_preprocess['valid_seen_{}'.format(pred_version)] = valid_seen_new_split
splits_to_preprocess['valid_unseen_{}'.format(pred_version)] = valid_unseen_new_split

print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
dataset = preprocess_language_predictions.Dataset(args, vocab=vocab)
preprocess_json_paths = dataset.preprocess_splits(splits_to_preprocess)

  1% (3 of 250) |                        | Elapsed Time: 0:00:00 ETA:   0:00:09


Preprocessing dataset and saving to pp folders ... This will take a while. Do this once as required.
Preprocessing train_sanity_v1


100% (250 of 250) |######################| Elapsed Time: 0:00:05 Time:  0:00:05
  2% (7 of 251) |                        | Elapsed Time: 0:00:00 ETA:   0:00:06

Preprocessing valid_seen_v1


100% (251 of 251) |######################| Elapsed Time: 0:00:05 Time:  0:00:05
  1% (4 of 255) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00

Preprocessing valid_unseen_v1


100% (255 of 255) |######################| Elapsed Time: 0:00:06 Time:  0:00:06


In [192]:
with open(os.path.join(args.split_dir, 'preprocessed_paths_{}.json'.format(pred_version)), 'w') as f:
    json.dump(preprocess_json_paths, f)

### Sanity Check

In [193]:
with open(os.path.join(args.split_dir, 'preprocessed_paths_{}.json'.format(pred_version)), 'r') as f:
    preprocess_json_paths = json.load(f)
    
preprocess_json_paths[:10]

['/root/data_alfred/json_feat_2.1.0/pick_and_place_simple-WineBottle-None-Shelf-7/trial_T20190907_200104_945677/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_cool_then_place_in_recep-Bowl-None-Cabinet-24/trial_T20190909_112909_923737/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-AppleSliced-None-DiningTable-17/trial_T20190908_095621_618627/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_and_place_simple-TennisRacket-None-SideTable-310/trial_T20190907_040920_796378/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_two_obj_and_place-CreditCard-None-Desk-313/trial_T20190909_104526_715846/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_and_place_simple-Candle-None-Cart-430/trial_T20190909_055319_119903/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-PotatoSliced-None-Fridge-22/trial_T20190908_185723_946017/pp/ann_3.json',
 '/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_

In [197]:
# machine
with open('/root/data_alfred/json_feat_2.1.0/pick_and_place_simple-Candle-None-Cart-430/trial_T20190909_055319_119903/pp/ann_3.json', 'r') as f:
    ex = json.load(f)
    
ex['num']['lang_instr']

[[47, 18, 4, 36, 18, 4, 19, 21, 4, 432, 16],
 [22, 23, 4, 492, 15, 4, 36, 16],
 [24, 25, 26, 47, 18, 4, 265, 16],
 [14, 4, 492, 15, 4, 66, 189, 21, 4, 160, 330, 16],
 [34]]

In [198]:
# human
with open('/root/data_alfred/json_feat_2.1.0/pick_and_place_simple-Candle-None-Cart-430/trial_T20190909_055319_119903/pp/ann_2.json', 'r') as f:
    ex = json.load(f)
    
ex['num']['lang_instr']

[[24, 18, 4, 19, 37, 24, 38, 26, 30, 18, 4, 36, 16],
 [22, 23, 4, 492, 6, 4, 36, 16],
 [24, 25, 26, 37, 24, 19, 16, 156, 18, 4, 259, 189, 549, 16],
 [10, 4, 492, 15, 4, 254, 21, 4, 259, 189, 16],
 [34]]